<a href="https://colab.research.google.com/github/vanshika1507/whisper_text/blob/main/assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --q git+https://github.com/m-bain/whisperx.git

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.7/208.7 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 7.1 MB/s eta 0:00

In [2]:
import whisperx
import gc

/usr/local/lib/python3.10/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


In [3]:
device = "cuda"
batch_size = 4 # reduce if low on GPU mem
compute_type = "float16" # change to "int8" if low on GPU mem (may reduce accuracy)

In [6]:
audio_file = "harvard.wav"

In [7]:
audio = whisperx.load_audio(audio_file)

In [8]:
model = whisperx.load_model("large-v2", device, compute_type=compute_type)


model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.80k [00:00<?, ?B/s]

No language specified, language will be first be detected for each audio file (increases inference time).


100%|█████████████████████████████████████| 16.9M/16.9M [00:02<00:00, 6.72MiB/s]
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.2.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.3.0+cu121. Bad things might happen unless you revert torch to 1.x.


In [9]:
result = model.transcribe(audio, batch_size=batch_size)
print(result["segments"]) # before alignment

# delete model if low on GPU resources
# import gc; gc.collect(); torch.cuda.empty_cache(); del model

# 2. Align whisper output
model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)


Detected language: en (1.00) in first 30s of audio...
[{'text': ' The stale smell of old beer lingers. It takes heat to bring out the odor. A cold dip restores health and zest. A salt pickle tastes fine with ham. Tacos al pastor are my favorite. A zestful food is the hot cross bun.', 'start': 1.254, 'end': 17.824}]


Downloading: "https://download.pytorch.org/torchaudio/models/wav2vec2_fairseq_base_ls960_asr_ls960.pth" to /root/.cache/torch/hub/checkpoints/wav2vec2_fairseq_base_ls960_asr_ls960.pth
100%|██████████| 360M/360M [00:04<00:00, 89.9MB/s]


In [10]:
result

{'segments': [{'start': 1.374,
   'end': 3.555,
   'text': ' The stale smell of old beer lingers.',
   'words': [{'word': 'The', 'start': 1.374, 'end': 1.494, 'score': 0.989},
    {'word': 'stale', 'start': 1.574, 'end': 1.914, 'score': 0.863},
    {'word': 'smell', 'start': 1.974, 'end': 2.335, 'score': 0.859},
    {'word': 'of', 'start': 2.375, 'end': 2.415, 'score': 1.0},
    {'word': 'old', 'start': 2.615, 'end': 2.775, 'score': 0.699},
    {'word': 'beer', 'start': 2.835, 'end': 3.075, 'score': 0.733},
    {'word': 'lingers.', 'start': 3.135, 'end': 3.555, 'score': 0.798}]},
  {'start': 4.476,
   'end': 6.257,
   'text': 'It takes heat to bring out the odor.',
   'words': [{'word': 'It', 'start': 4.476, 'end': 4.536, 'score': 0.844},
    {'word': 'takes', 'start': 4.596, 'end': 4.876, 'score': 0.877},
    {'word': 'heat', 'start': 4.956, 'end': 5.176, 'score': 0.861},
    {'word': 'to', 'start': 5.256, 'end': 5.356, 'score': 0.751},
    {'word': 'bring', 'start': 5.396, 'end': 5.5

In [12]:
diarize_model = whisperx.DiarizationPipeline(use_auth_token="HUGGINGFACE_TOKEN",
                                             device=device)


Could not download 'pyannote/speaker-diarization-3.1' pipeline.
It might be because the pipeline is private or gated so make
sure to authenticate. Visit https://hf.co/settings/tokens to
create your access token and retry with:

   >>> Pipeline.from_pretrained('pyannote/speaker-diarization-3.1',
   ...                          use_auth_token=YOUR_AUTH_TOKEN)

If this still does not work, it might be because the pipeline is gated:
visit https://hf.co/pyannote/speaker-diarization-3.1 to accept the user conditions.


AttributeError: 'NoneType' object has no attribute 'to'

In [ ]:
diarize_segments = diarize_model(audio, min_speakers=2, max_speakers=2)

In [ ]:
diarize_segments

,segment,label,speaker,start,end
0,[ 00:00:00.008 --> 00:00:00.025],A,SPEAKER_01,0.008489,0.025467
1,[ 00:00:00.195 --> 00:00:36.697],B,SPEAKER_01,0.195246,36.697793
2,[ 00:00:37.835 --> 00:00:38.327],C,SPEAKER_01,37.835314,38.327674
3,[ 00:00:39.159 --> 00:00:44.643],D,SPEAKER_01,39.159593,44.643463
4,[ 00:00:45.475 --> 00:00:48.022],E,SPEAKER_01,45.475382,48.022071
...,...,...,...,...,...
83,[ 00:07:39.142 --> 00:07:45.526],CF,SPEAKER_01,459.142615,465.526316
84,[ 00:07:46.986 --> 00:07:50.161],CG,SPEAKER_01,466.986418,470.161290
85,[ 00:07:50.738 --> 00:07:51.061],CH,SPEAKER_00,470.738540,471.061121
86,[ 00:07:51.061 --> 00:07:53.471],CI,SPEAKER_01,471.061121,473.471986


In [ ]:
diarize_segments.speaker.unique()

array(['SPEAKER_01', 'SPEAKER_00'], dtype=object)

In [ ]:
result = whisperx.assign_word_speakers(diarize_segments, result)
print(diarize_segments)
print(result["segments"]) # segments are now assigned speaker IDs

                              segment label     speaker       start  \
0   [ 00:00:00.008 -->  00:00:00.025]     A  SPEAKER_01    0.008489   
1   [ 00:00:00.195 -->  00:00:36.697]     B  SPEAKER_01    0.195246   
2   [ 00:00:37.835 -->  00:00:38.327]     C  SPEAKER_01   37.835314   
3   [ 00:00:39.159 -->  00:00:44.643]     D  SPEAKER_01   39.159593   
4   [ 00:00:45.475 -->  00:00:48.022]     E  SPEAKER_01   45.475382   
..                                ...   ...         ...         ...   
83  [ 00:07:39.142 -->  00:07:45.526]    CF  SPEAKER_01  459.142615   
84  [ 00:07:46.986 -->  00:07:50.161]    CG  SPEAKER_01  466.986418   
85  [ 00:07:50.738 -->  00:07:51.061]    CH  SPEAKER_00  470.738540   
86  [ 00:07:51.061 -->  00:07:53.471]    CI  SPEAKER_01  471.061121   
87  [ 00:07:53.777 -->  00:07:58.463]    CJ  SPEAKER_01  473.777589   

           end  intersection       union  
0     0.025467   -478.264533  478.361511  
1    36.697793   -441.592207  478.174754  
2    38.327674   -

In [11]:
result

{'segments': [{'start': 1.374,
   'end': 3.555,
   'text': ' The stale smell of old beer lingers.',
   'words': [{'word': 'The', 'start': 1.374, 'end': 1.494, 'score': 0.989},
    {'word': 'stale', 'start': 1.574, 'end': 1.914, 'score': 0.863},
    {'word': 'smell', 'start': 1.974, 'end': 2.335, 'score': 0.859},
    {'word': 'of', 'start': 2.375, 'end': 2.415, 'score': 1.0},
    {'word': 'old', 'start': 2.615, 'end': 2.775, 'score': 0.699},
    {'word': 'beer', 'start': 2.835, 'end': 3.075, 'score': 0.733},
    {'word': 'lingers.', 'start': 3.135, 'end': 3.555, 'score': 0.798}]},
  {'start': 4.476,
   'end': 6.257,
   'text': 'It takes heat to bring out the odor.',
   'words': [{'word': 'It', 'start': 4.476, 'end': 4.536, 'score': 0.844},
    {'word': 'takes', 'start': 4.596, 'end': 4.876, 'score': 0.877},
    {'word': 'heat', 'start': 4.956, 'end': 5.176, 'score': 0.861},
    {'word': 'to', 'start': 5.256, 'end': 5.356, 'score': 0.751},
    {'word': 'bring', 'start': 5.396, 'end': 5.5